<img src="./img/logos_combined.png" align="right" width="50%"></img>
<br><br><br>

# Python's OWSlib library - Using OGC web services from Python

<hr>
<a href="./index.ipynb"><< Index</a>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space>&nbsp;<space> <a href="./02_wcps_example.ipynb">WCPS example >></a>

The OWSLib library is a python library that makes accessing data and metadata through OGC services possible. The library abstracts all of the traditional HTTP API calls into programmatic function calls. Two examples are given: one how to use the WebMapService class and one how to use the WebCoverageService class.

## OWSlib library - Using WMS from Python

#### <i>Create a WebMapService object</i>

In [ ]:
# load owslib library
from owslib.wms import WebMapService

# Create your WebMapService object
wms = WebMapService('http://apps.ecmwf.int/wms/?token=public', version='1.3.0')

#### <i>Request layer information</i>

In [ ]:
# See available layers
list(wms.contents)

#### <i>Get layer description</i>

In [ ]:
wms['composition_co2_surface'].title

#### <i>Get bounding box information in WGS84</i>

In [ ]:
wms['composition_co2_surface'].boundingBoxWGS84

#### <i>Get available styles</i>

In [ ]:
wms['composition_co2_surface'].styles

#### <i>See available methods</i>

In [ ]:
[op.name for op in wms.operations]

#### <i>Get available formats</i>

In [ ]:
wms.getOperationByName('GetMap').formatOptions

#### <i>Request a map</i>

In [ ]:
%matplotlib inline
import os, sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

def getMap(layerName,bbox,filename):
    wms.getOperationByName('GetMap').formatOptions
    img = wms.getmap(layers=[layerName],
                 size=(600,300),
                 srs='EPSG:4326',
                 bbox=bbox,
                 format='image/png',
                 transparent=True)

    tmpfile = open(filename,'wb')
    tmpfile.write(img.read())
    tmpfile.close()
getMap('foreground',(-180,-90,180,90), 'foreground.png')
getMap('background',(-180,-90,180,90), 'background.png')
getMap('composition_bbaod550',(-180,-90,180,90), 'bbaod550.png')


image1=mpimg.imread('background.png')
image2=mpimg.imread('bbaod550.png')
image3=mpimg.imread('foreground.png')
fig = plt.figure(figsize=(12,7))

img1=plt.imshow(image1,extent=[-180,180,-90,90],aspect='auto')
img2=plt.imshow(image2,extent=[-180,180,-90,90],aspect='auto')
img3=plt.imshow(image3,extent=[-180,180,-90,90],aspect='auto')
plt.show()

<hr>

## OWSlib library - Using WCS from Python

The library supporting WCS 2.0 is available from GitHub @ https://github.com/earthserver-eu/OWSLib

#### <i>Create a WebCoverageService (WCS) object</i>

In [ ]:
from owslib.wcs import WebCoverageService

ecmwf_wcs = WebCoverageService('http://earthserver.ecmwf.int/rasdaman/ows?', version='2.0.1')

In [ ]:
print(ecmwf_wcs)

#### <i>Get a list of available coverages</i>

In [ ]:
for coverage_name in ecmwf_wcs.contents.keys():
    print(coverage_name)

#### <i>Retrieve a list of available properties of a coverage</i>

In [ ]:
for item in dir(ecmwf_wcs.contents['temp2m']):
    if "_" not in item:
        print(item)

#### <i>Retrieve bounding box information</i>

In [ ]:
ecmwf_wcs.contents['temp2m'].boundingboxes

#### <i>Retrieve grid property information</i>

In [ ]:
for item in dir(ecmwf_wcs.contents['temp2m'].grid):
    if "_" not in item:
        print (item + ": " + str(ecmwf_wcs.contents['temp2m'].grid.__dict__[item]))

#### <i>Retrieve supported formats</i>

In [ ]:
ecmwf_wcs.contents['temp2m'].supportedFormats

#### <i>Build a GetCoverage request based on the collected coverage information</i>

In [ ]:
# Request a global 2m air temperature data field from 1 August 2003 at 12:00 UTC as netcdf.
# Store the file temporarily and load it for plotting of the data field

%matplotlib inline
import netCDF4 as nc
import numpy as np
from matplotlib import pyplot as plt

# GetCoverage request
coverage_file = ecmwf_wcs.getCoverage(identifier=['temp2m'], format='application/netcdf', 
                                      subsets=[('Long',-180,179.5),('Lat',-90,90),('ansi',"2003-08-01T12:00","2003-08-01T18:00")])

# Store the requested field as netCDF file
with open('testCoverage1.nc','w') as outfile:
    outfile.write(coverage_file.read())

# Open the stored netCDF file
ncdata = nc.Dataset("testCoverage1.nc","a",format="NETCDF4")

# Retrieve the data from the netCDF object
data = np.flipud(np.rot90(ncdata.variables['field_1'][:,:,0]))

# Plot the data
plt.figure(figsize=(40,20))
plt.imshow(data)

<hr>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>